# Compute SNR for galactic binaries

### Imports and constants

In [ ]:
from gbgpu import gbgpu
import pandas as pd
import numpy as np

# useful imports
from lisatools.sensitivity import LISASens, get_sensitivity, get_stock_sensitivity_options
from lisatools.sensitivity import SensitivityMatrix, LISASens, A1TDISens, E1TDISens
import lisatools.detector as lisa_models
import matplotlib.pyplot as plt

from lisatools.utils.constants import YRSID_SI
from tqdm import tqdm
import glob

YEAR = 525600 * 60

### Make the noise PSD model

In [2]:
time = np.arange(0, YEAR, 5.0)
f = np.fft.rfftfreq(len(time), d=5.0)[1:]
data = [np.zeros(len(f)), np.zeros(len(f))]
Sn = get_sensitivity(f, sens_fn=LISASens, average=True, model=lisa_models.sangria, return_type='PSD')

sens_kwargs = dict(
    stochastic_params=(1.0 * YRSID_SI,)
)

sens_mat = SensitivityMatrix(f, [A1TDISens, E1TDISens], **sens_kwargs)
sens_mat.update_model(lisa_models.sangria)

### Instantiate the waveform model

In [3]:
gb = gbgpu.GBGPU()

### Read in the feather files with parameter values

In [4]:
df = pd.read_feather('/Users/aaron/Documents/lisa/catalog_metrics/data/igb_params.feather')

In [5]:
# i = 0

# amp = df['Amplitude'].values[1]
# f0 = df['Frequency'].values[1]
# fdot = df['FrequencyDerivative'].values[1]
# fddot = 0.0
# phi0 = df['InitialPhase'].values[1]
# iota = df['Inclination'].values[1]
# psi = df['Polarization'].values[1]
# lam = df['EclipticLongitude'].values[1]
# beta = df['EclipticLatitude'].values[1]

# params = np.array([amp, f0, fdot, fddot, phi0, iota, psi, lam, beta])
# params.reshape((1, -1))
# gb.d_d = 0.0
# np.real(gb.get_ll(params, data, [Sn, Sn]))

### Iterate over the parameter values to compute SNRs
* Note: GBGPU has been modified so that gb.get_ll() returns h_h instead of the loglikelihood
* It's easy enough to modify this to instead compute gb.get_ll() and then take gb.h_h as the result

In [10]:
# GBGPU has had the output of gb.get_ll changed to output gb.h_h here
# iterate through the dataframe and set an SNR parameter
names = []
snr_values = []
start_index = 0
for i in tqdm(range(start_index, len(df))):

    name = df['Name'].values[i]
    amp = df['Amplitude'].values[i]
    f0 = df['Frequency'].values[i]
    fdot = df['FrequencyDerivative'].values[i]
    fddot = 0.0
    phi0 = df['InitialPhase'].values[i]
    iota = df['Inclination'].values[i]
    psi = df['Polarization'].values[i]
    lam = df['EclipticLongitude'].values[i]
    beta = df['EclipticLatitude'].values[i]

    # print(amp, f0, fdot, fddot, phi0, iota, psi, lam, beta)

    params = np.array([amp, f0, fdot, fddot, phi0, iota, psi, lam, beta])
    params.reshape((1, -1))
    gb.d_d = 0.0
    names.append(name)
    snr_values.append(np.real(gb.get_ll(params, data, sens_mat)[0]))
    # if i in [8162]:
    #     names.append(name)
    #     snr_values.append(0)
    #     continue
    if len(snr_values) % 10_000 == 0:
        np.savetxt(f'./snrs/igb_snrs/snr_values_{i // 10_000}.txt', np.c_[names, snr_values])
        names = []
        snr_values = []
# np.savetxt('./snrs/snr_values_final.txt', np.c_[names, snr_values])
# snr_values = []


100%|██████████| 3000000/3000000 [19:48<00:00, 2524.75it/s]


In [ ]:
df = pd.read_feather('/Users/aaron/Documents/lisa/catalog_metrics/data/dgb_params.feather')
dgb_snrs = glob.glob('./snrs/dgb_snrs/snr_values_*.txt')

In [62]:
# build a single mapping of Name → SNR
all_names = []
all_snr_values = []
mapping = {}
for file in dgb_snrs:
    names, snr_values = np.loadtxt(file, unpack=True, dtype=np.float64)
    all_names.append(np.array(names).astype(np.float64))
    all_snr_values.append(np.array(snr_values).astype(np.float64))
all_names = np.concatenate(all_names)
all_snr_values = np.concatenate(all_snr_values)
print(len(all_names))
print(len(all_snr_values))
# suppose you’ve concatenated all your names & snr_values into two lists:
#   all_names, all_snr_values
snr_df = pd.DataFrame({'Name': all_names, 'SNR': all_snr_values})

# ensure your main df['Name'] is also float64

df = df.merge(snr_df, on='Name')

26000000
26000000


In [ ]:
# df2 = pd.read_feather('/Users/aaron/Documents/lisa/catalog_metrics/data/igb_params.feather')
# igb_snrs = glob.glob('./snrs/igb_snrs/snr_values_*.txt')

In [ ]:
# # build a single mapping of Name → SNR
# all_names = []
# all_snr_values = []
# mapping = {}
# for file in igb_snrs:
#     names, snr_values = np.loadtxt(file, unpack=True, dtype=np.float64)
#     all_names.append(np.array(names).astype(np.float64))
#     all_snr_values.append(np.array(snr_values).astype(np.float64))
# all_names = np.concatenate(all_names)
# all_snr_values = np.concatenate(all_snr_values)
# print(len(all_names))
# print(len(all_snr_values))
# # suppose you’ve concatenated all your names & snr_values into two lists:
# #   all_names, all_snr_values
# snr_df = pd.DataFrame({'Name': all_names, 'SNR': all_snr_values})

# # ensure your main df['Name'] is also float64

# df2 = df2.merge(snr_df, on='Name')

3000000
3000000
